<a href="https://colab.research.google.com/github/surabhipandey18/FoodAgri/blob/main/fertilizer_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, Dense, Dropout
from tensorflow.keras.metrics import CategoricalAccuracy

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Hackathon/f2.csv')
df.head()

,Temparature,Humidity,Moisture,Soil_Type,Crop_Type,Nitrogen,Potassium,Phosphorous,Fertilizer
0,20,83,26,Clayey,rice,90,49,36,Urea
1,25,84,32,Loamy,rice,66,59,36,Urea
2,33,64,50,Loamy,Wheat,41,0,0,Urea
3,34,65,54,Loamy,Wheat,38,0,0,Urea
4,38,72,51,Loamy,Wheat,39,0,0,Urea


In [3]:
df.shape

(552, 9)

In [4]:
df = df.dropna()

In [65]:
categorical_col = ['Soil_Type', 'Crop_Type']
numerical_col = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
y = df['Fertilizer']
df.shape

(552, 9)

In [66]:
X = df[categorical_col + numerical_col]

In [67]:
std = StandardScaler()
encoder = OneHotEncoder()

In [68]:
y = encoder.fit_transform(y.values.reshape(-1, 1)).toarray()

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.2, random_state=42)

In [70]:
preprocess = ColumnTransformer([
    ('std', std, numerical_col),
    ('encoder', encoder, categorical_col)
])

In [71]:
preprocess.fit(X_train)
X_train = preprocess.transform(X_train)
X_test = preprocess.transform(X_test)

In [72]:
X_test.shape

(111, 28)

In [74]:
num_classes = y_train.shape[1]
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[CategoricalAccuracy()])
tf.config.run_functions_eagerly(True)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 128)            │         3,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 14)             │           462 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,510 (56.68 KB)

 Trainable params: 14,510 (56.68 KB)

 Non-trainable params: 0 (0.00 B)

In [75]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - categorical_accuracy: 0.1150 - loss: 2.6065 - val_categorical_accuracy: 0.1685 - val_loss: 2.4620
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - categorical_accuracy: 0.2670 - loss: 2.3877 - val_categorical_accuracy: 0.2135 - val_loss: 2.3016
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - categorical_accuracy: 0.2599 - loss: 2.2463 - val_categorical_accuracy: 0.3146 - val_loss: 2.1289
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 149ms/step - categorical_accuracy: 0.3650 - loss: 2.0467 - val_categorical_accuracy: 0.3483 - val_loss: 1.9390
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 102ms/step - categorical_accuracy: 0.3671 - loss: 1.8252 - val_categorical_accuracy: 0.3708 - val_loss: 1.7289
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - categorical_accuracy: 0.4654 - loss: 1.5826 - val_categorical_accuracy: 0.4045 - val_loss: 1.5178
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - categorical_accuracy: 0.4983 - loss: 1.4403 - 

In [76]:
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")
val_accuracy = model.evaluate(X_test, y_test)[1]
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - categorical_accuracy: 0.9855 - loss: 0.0783
Test Accuracy: 98.20%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - categorical_accuracy: 0.9855 - loss: 0.0783
Validation Accuracy: 98.20%


In [77]:
model.evaluate(X_test, y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - categorical_accuracy: 0.9855 - loss: 0.0783


[0.07732151448726654, 0.9819819927215576]

In [78]:
model.save('fertilizer_recommendation.keras')